In [3]:
import pyrootutils
path = pyrootutils.find_root(search_from="./", indicator=".home")
pyrootutils.set_root(
    path=path, # path to the root directory
    project_root_env_var=True, # set the PROJECT_ROOT environment variable to root directory
    dotenv=True, # load environment variables from .env if exists in root directory
    pythonpath=True, # add root directory to the PYTHONPATH (helps with imports)
    cwd=True, # change current working directory to the root directory (helps with filepaths)
)


PosixPath('/Users/eman/code_projects/oceanbench')

In [7]:
from oceanbench._src.datasets.base import XRDataset, XRConcatDataset
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from 


In [66]:
x_coord = Coordinate(40, 50, 0.05, "X")
y_coord = Coordinate(-60, -50, 0.05, "Y")

grid = Grid2D(x_coord, y_coord)
grid.grid.shape, grid.coords.shape


((201, 201, 2), (201, 2))

In [28]:
n_days = 10
delta = "1D"
days =  np.arange(0, n_days, 1)
time = pd.to_datetime('2020-01-01') + days *  pd.to_timedelta(delta)
n_degrees = 10
lon = 40 + np.arange(0, n_degrees+1, 1)
lat = -60 + np.arange(0, n_degrees+1, 1)

In [12]:
time

Timestamp('2020-02-20 00:00:00')

In [129]:
from xarray_dataclasses import Coordof
from typing import Literal
from xarray_dataclasses import AsDataArray, AsDataset
from xarray_dataclasses import Attr, Coord, Data, Name, asdataarray, asdataset

X = tp.Literal["x"]
LAT = tp.Literal["lat"]
LON = tp.Literal["lon"]
TIME = tp.Literal["time"]

@dataclass
class CoordinateAxis:
    data: Data[X, np.ndarray]
    long_name: Attr[str] = ""
    units: Attr[str] = ""

    @classmethod
    def init_from_bounds(cls, x_min: float, x_max: float, dx: float):
        data = np.arange(x_min, x_max + dx, dx)
        return cls(data=data)

@dataclass
class LongitudeAxis(CoordinateAxis):
    data: Data[LON, np.ndarray]
    name: Name[str] = "lon"
    standard_name: Attr[str] = "longitude"
    long_name: Attr[str] = "Longitude"
    units: Attr[str] = "degrees_east"

@dataclass
class LatitudeAxis(CoordinateAxis):
    data: Data[LAT, np.ndarray]
    name: Name[str] = "lat"
    standard_name: Attr[str] = "latitude"
    long_name: Attr[str] = "Latitude"
    units: Attr[str] = "degrees_west"

@dataclass
class TimeAxis:
    data: Data[TIME, tp.Literal["datetime64[ns]"]]
    name: Name[str] = "time"
    long_name: Attr[str] = "Date"

    @classmethod
    def init_from_bounds(cls, t_min: str, t_max: str, dt: str):
        t_min = pd.to_datetime(t_min)
        t_max = pd.to_datetime(t_max)
        dt = pd.to_timedelta(dt)
        data = np.arange(t_min, t_max + dt, dt)
        return cls(data=data)


@dataclass
class SSH2D(AsDataArray):
    data: Data[tuple[LON, LAT], np.ndarray]
    lon: Coordof[LongitudeAxis] = 0
    lat: Coordof[LatitudeAxis] = 0
    name: Name[str] = "ssh"
    units: Attr[str] = "m"
    standard_name: Attr[str] = "sea_surface_height"
    long_name: Attr[str] = "Sea Surface Height"

@dataclass
class SSH2DT(AsDataArray):
    data: Data[tuple[TIME, LAT, LON], np.ndarray]
    lon: Coordof[LongitudeAxis] = 0
    lat: Coordof[LatitudeAxis] = 0
    time: Coordof[TimeAxis] = 0
    name: Name[str] = "ssh"
    units: Attr[str] = "m"
    standard_name: Attr[str] = "sea_surface_height"
    long_name: Attr[str] = "Sea Surface Height"

In [130]:
def create_coord_grid(*args):

    grid = np.meshgrid(*args, indexing="ij")

    return np.stack(grid, axis=-1)

In [131]:
lon_axis = LongitudeAxis.init_from_bounds(x_min=40, x_max=50, dx=0.05)
lat_axis = LatitudeAxis.init_from_bounds(x_min=-60, x_max=-50, dx=0.05)

grid = create_coord_grid(lon_axis.data, lat_axis.data)

ssh_init = np.ones_like(grid[...,-1])
ssh_data = SSH2D(data=ssh_init, lon=lon_axis, lat=lat_axis)
ssh_data = asdataarray(ssh_data)
ssh_data

<xarray.DataArray 'ssh' (lon: 201, lat: 201)>
array([[1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       ...,
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
       [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0]], dtype=object)
Coordinates:
  * lon      (lon) object 40.0 40.05 ... 49.949999999999434 49.99999999999943
  * lat      (lat) object -60.0 -59.95 ... -50.00000000000057
Attributes:
    units:          m
    standard_name:  sea_surface_height
    long_name:      Sea Surface Height

In [127]:
grid[0].dtype, grid[1].dtype, grid[2].dtype

(dtype('float64'), dtype('float64'), dtype('float64'))

In [128]:
lon_axis = LongitudeAxis.init_from_bounds(x_min=40, x_max=50, dx=0.05)
lat_axis = LatitudeAxis.init_from_bounds(x_min=-60, x_max=-50, dx=0.05)
time_axis = TimeAxis.init_from_bounds(t_min="2020-01-01", t_max="2020-01-30", dt="1D")


ssh_init = np.ones((
    time_axis.data.shape[0],
     lat_axis.data.shape[0],
     lon_axis.data.shape[0]
    ))
ssh_data = SSH2DT(data=ssh_init, lon=lon_axis, lat=lat_axis, time=time_axis)
ssh_data = asdataarray(ssh_data)
ssh_data

<xarray.DataArray 'ssh' (t: 30, lat: 201, lon: 201)>
array([[[1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        ...,
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0]],

       [[1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        ...,
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0]],

       [[1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        ...,
...
        ...,
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0]],

       [[1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        ...,
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0]],

       [[1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        ...,
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0]]], dtype=object)
Coordinates:
  * lon      (lon) object 40.0 40.05 ... 49.949999999999434 49.99999999999943
  * lat      (lat) object -60.0 -59.95 ... -50.00000000000057
    time     (t) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-29 2020-01-30
Dimensions without coordinates: t
Attributes:
    units:          m
    standard_name:  sea_surface_height
    long_name:      Sea Surface Height